In [24]:
import requests
import sys
import pandas as pd
import numpy as np
import time
import re
# time.sleep(5)
# page = requests.get('https://www.boxofficemojo.com/release/rl1182631425/?ref_=bo_yld_table_1')
# page_list = requests.get('https://www.boxofficemojo.com/year/2020/?grossesOption=calendarGrosses')
from bs4 import BeautifulSoup
# soup = BeautifulSoup(page.content, 'html.parser')
# print(soup.prettify())
# page = requests.get('https://www.boxofficemojo.com/release/rl2960360961/?ref_=bo_yld_table_16')
# page = requests.get('https://www.boxofficemojo.com/release/rl101418497/?ref_=bo_yld_table_235')
time.sleep(1)
# soup = BeautifulSoup(page.content, 'html.parser')    

In [2]:
# returns the title of the single movie from single movie url in string form
def find_movie_title(page):
    # page = requests.get('https://www.boxofficemojo.com/release/rl2960360961/?ref_=bo_yld_table_16')
    # time.sleep(1)
    soup = BeautifulSoup(page.content, 'html.parser')    
    # print(soup.prettify())
    title = soup.find('h1', class_="a-size-extra-large")
    return title.string

In [3]:
find_movie_title(page)

'Killing Sarai'

In [4]:
# returns release date of the single movie in string
def find_release_date(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    release_date = soup.find_all('a', class_='a-link-normal')
    date = ''
    for item in release_date:
        if item['href'][-8:] == 'bo_rl_rl':
            # print('here')
            date = item.text
    
    return date

In [5]:
find_release_date(page)

'Jan 3, 2020'

In [6]:
# returns budget cost or NaN of a movie if no budget cost is listed
def find_budget(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    budget = str(soup.find_all('div', class_='a-section a-spacing-none'))
    # print(budget)
    re_budget = re.findall(r"(?<=Budget</span><span><span class=\"money\">).*?(?=</span>)", budget)
    if re_budget:
        return re_budget[0]
    # print(budget)
    return float('NaN')


In [7]:
find_budget(page)

nan

In [8]:
# returns an array of genres of single movie
def find_genre(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    span_genre = str(soup.find_all('span')).replace('\n', '')
    span_genre = re.findall(r"(?<=<span>Genres<\/span>, <span>).*?(?=</span>)", span_genre)
    return span_genre[0].split()

In [9]:
find_genre(page)

['Romance', 'Thriller']

In [10]:
# returns gross value (domestic and world wide) in dict with two key-values
def find_all_gross(page):
    output = []
    soup = BeautifulSoup(page.content, 'html.parser') 
    all_money = soup.select('.a-link-normal > .money')
    if (len(all_money) == 2):
        international = int(all_money[0].text.replace('$', '').replace(',', ''))
        worldwide = int(all_money[1].text.replace('$', '').replace(',', ''))
        domestic = worldwide - international
        return {'domestic_gross': domestic, 'worldwide_gross': worldwide}
    if (len(all_money) < 2):
        worldwide = int(all_money[0].text.replace('$', '').replace(',', ''))
        return {'domestic_gross': worldwide, 'worldwide_gross': worldwide}
    else:
        return {'domestic_gross': float('NaN'), 'world_gross': float('NaN')}

In [11]:
find_all_gross(page)

{'domestic_gross': 425332, 'worldwide_gross': 425332}

In [12]:
# returns MPAA rating in string if not found, return nan
def find_mpaa(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    # print(soup)
    mpaa_rating = str(soup.find_all('div', class_='a-section a-spacing-none'))
    mpaa_rating = re.findall(r"(?<=<span>MPAA</span><span>).*?(?=</span>)", mpaa_rating)
    # print(mpaa_rating)
    if mpaa_rating:
        return mpaa_rating[0]
    return float('nan')

In [13]:
find_mpaa(page)

'R'

In [46]:
# create a function that will return a dictionary of a single movie with keys as 'title', 'release_date', 
# 'genres', 'mpaa', 'budget_cost', 'total_gross'
def create_single_movie_dict(single_movie_url):
    new_movie = {}
    # print(single_movie_url)
    time.sleep(.5)

    page = requests.get(single_movie_url)
    # print('here', page.content)
    soup = BeautifulSoup(page.content, 'html.parser')    
    # print(soup.prettify())
    title = find_movie_title(page)
    print(title)
    release_date = find_release_date(page)
    print(release_date)
    genres = find_genre(page)
    print(genres)
    mpaa = find_mpaa(page)
    print(mpaa)
    budget_cost = find_budget(page)
    print(budget_cost)
    all_gross = find_all_gross(page)
    print(all_gross)
    # assign into dict
    if title not in  new_movie:
        new_movie['title'] = title
        # print(title)
        new_movie['release_date'] = release_date
        new_movie['genres'] = genres
        new_movie['mpaa'] = mpaa
        new_movie['budget_cost'] = budget_cost
        new_movie['total_gross_worldwide'] = all_gross
    return new_movie

In [47]:
create_single_movie_dict('https://www.boxofficemojo.com/release/rl2747040257/weekend/')

Moondance
Mar 13, 2020
['Musical']
nan
nan
{'domestic_gross': 2371, 'worldwide_gross': 2371}


{'title': 'Moondance',
 'release_date': 'Mar 13, 2020',
 'genres': ['Musical'],
 'mpaa': nan,
 'budget_cost': nan,
 'total_gross_worldwide': {'domestic_gross': 2371, 'worldwide_gross': 2371}}